In [1]:
import os
import pandas as pd
from rdflib import Graph, Literal, URIRef

In [2]:
data_path = 'C:\\Users\\Nicolas\\Desktop\\Nicolas\\DAT475-labs\\Data'
uri_prefix = 'http://www.semanticweb.org/aapohaavisto/ontologies/2024/3/untitled-ontology-2#'

### Import tabular data

In [3]:
assigned_hours = pd.read_csv(os.path.join(data_path, 'Assigned_Hours.csv'))
course_instances = pd.read_csv(os.path.join(data_path, 'Course_Instances.csv'))
course_plannings = pd.read_csv(os.path.join(data_path, 'Course_plannings.csv'))
courses = pd.read_csv(os.path.join(data_path, 'Courses.csv'))
programme_courses = pd.read_csv(os.path.join(data_path, 'Programme_Courses.csv'))
programmes = pd.read_csv(os.path.join(data_path, 'Programmes.csv'))
registrations = pd.read_csv(os.path.join(data_path, 'Registrations.csv'))
reported_hours = pd.read_csv(os.path.join(data_path, 'Reported_Hours.csv'))
senior_teachers = pd.read_csv(os.path.join(data_path, 'Senior_Teachers.csv'))
students = pd.read_csv(os.path.join(data_path, 'Students.csv'))
teaching_assistants = pd.read_csv(os.path.join(data_path, 'Teaching_Assistants.csv'))

### Create data tables for the new entities

In [4]:
# Only teachers and its subclass senior_teachers will be used (as in ER)
teachers = pd.concat([senior_teachers, teaching_assistants])
teachers.head()

,Teacher name,Teacher id,Department name,Division name
0,Teacher 1,19630126-0001,D4,D4-3
1,Teacher 2,19600814-0002,D7,D7-4
2,Teacher 3,19600905-0003,D7,D7-1
3,Teacher 4,19590129-0004,D3,D3-3
4,Teacher 5,19610623-0005,D5,D5-2


In [5]:
# Create division table, where 'Department name' is a foreign key
divisions = teachers[['Division name', 'Department name']]
divisions = divisions.drop_duplicates(ignore_index=True)  # we only want unique entries here
divisions.head()

,Division name,Department name
0,D4-3,D4
1,D7-4,D7
2,D7-1,D7
3,D3-3,D3
4,D5-2,D5


In [6]:
# Extract departments from divisions
departments = divisions[['Department name']]
departments = departments.drop_duplicates(ignore_index=True)  # we only want unique entries here
departments.head()

,Department name
0,D4
1,D7
2,D3
3,D5
4,D6


In [7]:
# Join assigned and reported hours (the key was renamed in Reported_Hours.csv)
assigned_hours = assigned_hours.drop(columns = ['Course code', 'Study Period', 'Academic Year'])   # redundant, info is already in course_instances
teacher_assignments = assigned_hours.merge(reported_hours, how='left', on=['Course Instance', 'Teacher Id'])
teacher_assignments.head()

,Teacher Id,Assigned Hours,Course Instance,Reported Hours
0,19620522-0023,240.0,I-1,227.0
1,19790708-0041,100.0,I-1,115.0
2,19930221-0139,140.0,I-1,157.0
3,19690408-0009,280.0,I-2,281.0
4,19750220-0037,100.0,I-2,115.0


In [8]:
# Join course instances and course plannings
course_plannings = course_plannings.rename(columns={'Course': 'Instance_id'})
course_instances = course_instances.merge(course_plannings, how='left', on='Instance_id')
course_instances.head()

,Course code,Study period,Academic year,Instance_id,Examiner,Planned number of Students,Senior Hours,Assistant Hours
0,1204,1.0,2013-2014,I-1,19610918-0027,50,280,300
1,1082,3.0,2013-2014,I-2,19620522-0023,20,240,240
2,1190,4.0,2013-2014,I-3,19560705-0025,20,240,300
3,1009,3.0,2013-2014,I-4,19600529-0021,50,280,420
4,1264,4.0,2013-2014,I-5,19570828-0008,60,280,300


### Define mappings

In [9]:
# maps csv column names to attribute names used in ontology
atr_map = {
    'Department': 'dept',
    'Division': 'div',
    'Department name': 'dept',
    'Division name': 'div',
    'Teacher name': 'name',
    'Teacher id': 'teacherID',
    'Programme name': 'programmeName',
    'Programme code': 'programmeCode',
    'Course name': 'courseName',
    'Course code': 'courseCode',
    'Course': 'courseCode',
    'Credits': 'credits',
    'Level': 'level',
    'Study Year': 'studyYear',
    'Academic Year': 'academicYear',
    'Academic year': 'academicYear',
    'Course Type': 'courseType',
    'Study period': 'studyPeriod',
    'Instance_id': 'instanceID',
    'Course Instance': 'instanceID',
    'Planned number of Students': 'planningNumStudents',
    'Senior Hours': 'seniorHours',
    'Assistant Hours': 'assistantHours',
    'Assigned Hours': 'assignedHours',   # this column name actually modified in csv, due to ambiguity of 'Hours'
    'Reported Hours': 'reportedHours',   # ^^
    'Student id': 'studentID',
    'Status': 'status',
    'Grade': 'grade',
    'Student name': 'name',
    'Year': 'year',
    'Graduated': 'graduated'
}

In [10]:
# maps foreign keys to (relation, foreign entity)
teacher_assignments_fkeys = {
    'Teacher Id': ('AssignedOn', 'CourseInstance'),
    'Course code': ('AssignedTo', 'Teacher')
}
programme_courses_fkeys = {
    'Course code': ('CourseIn', 'Course'),
    'Programme code': ('ProgrammeIn', 'Programme')
}
registrations_fkeys = {
    'Course Instance': ('CourseRegistered', 'CourseInstance'),
    'Student id': ('StudentRegistered', 'Student')
}
programmes_fkeys = {
    'Director': ('HasDirector', 'SeniorTeacher'),
    'Department name': ('OfDepartment', 'Department')
}
course_instances_fkeys = {
    'Examiner': ('HasExaminer', 'SeniorTeacher'),
    'Course code': ('InstanceOf', 'Course')
}
courses_fkeys = {
    'Owned By': ('HasOwner', 'Programme'),
    'Division': ('OfDivision', 'Division')
}
students_fkeys = {
    'Programme': ('InProgramme', 'Programme')
}
divisions_fkeys = {
    'Department name': ('OfDepartment', 'Department')
}
teachers_fkeys = {
    'Division name': ('OfDivision', 'Division')
}

### Convert tabular data to graphs

In [11]:
# Relies on correct mappings above
def add_triples(graph, data, entity_name, primary_keys, foreign_keys):
    entity_uri = uri_prefix + entity_name
    for _, row in data.iterrows():
        uid = '-'.join(str(row[pk]) for pk in primary_keys)   # unique identifier is simply value of pk, or pk1-pk2-pk3... if more than one
        instance_uri = URIRef(entity_uri + '/' + uid)
        for col in row.keys():
            if col in foreign_keys.keys():
                relationship_name = foreign_keys[col][0]
                fentity_name = foreign_keys[col][1]
                relationship_uri = URIRef(uri_prefix + relationship_name)
                finstance_uri = URIRef(uri_prefix + fentity_name + '/' + str(row[col]))
                graph.add((instance_uri, relationship_uri, finstance_uri))    # connect instance to foreign instance
            else:
                dprop_uri = URIRef(uri_prefix + atr_map[col])
                graph.add((instance_uri, dprop_uri, Literal(row[col])))   # add data property value to instance

In [12]:
# Create graph and add triples
g = Graph()

add_triples(g, departments, 'Department', ['Department name'], {})
add_triples(g, divisions, 'Division', ['Division name'], divisions_fkeys)
add_triples(g, teacher_assignments, 'TeacherAssignment', ['Course Instance', 'Teacher Id'], teacher_assignments_fkeys)
add_triples(g, course_instances, 'CourseInstance', ['Instance_id'], course_instances_fkeys)
add_triples(g, courses, 'Course', ['Course code'], courses_fkeys)
add_triples(g, programme_courses, 'ProgrammeCourse', ['Programme code', 'Study Year', 'Academic Year'], programme_courses_fkeys)
add_triples(g, programme_courses, 'Programme', ['Programme code'], programmes_fkeys)
add_triples(g, registrations, 'Registration', ['Course Instance', 'Student id'], registrations_fkeys)
add_triples(g, teachers, 'Teacher', ['Teacher id'], teachers_fkeys)
add_triples(g, senior_teachers, 'SeniorTeacher', ['Teacher id'], {})  # NB: How to model ISA data? Now it is just dupes of subset of teachers
add_triples(g, students, 'Student', ['Student id'], students_fkeys)  

# Save graph
g.serialize('data.rdf', format='xml')
g.serialize('data.ttl.txt', format='turtle')

<Graph identifier=Nc63c5abfa2bb4461befbc175ebd14b6b (<class 'rdflib.graph.Graph'>)>